<center>

# $\textbf{Gross Domestic Product }$

<center>

### $\textbf{Code}$

In [60]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, lit, array, struct

In [61]:
spark = SparkSession.builder.appName('GDP').master("local").enableHiveSupport().getOrCreate()
spark

In [62]:
# Creating dataframe from the csv file and infering the schema
df = spark.read.load("Files/GDP.csv", format="csv", sep=",", inferschema="true", header="true")

In [63]:
# Assuming df is properly defined DataFrame
df = df.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("gdp")) 
    for year in range(2000, 2025)
])).alias("data")).selectExpr("country", "data.year", "data.gdp")

In [64]:
#Storing this dataframe in parquet
df.write.mode("overwrite").parquet("FilesParquet/GDP.parquet" )
spark.read.parquet("FilesParquet/GDP.parquet").show()
spark.stop()

+-----------+----+-------+
|    country|year|    gdp|
+-----------+----+-------+
|Afghanistan|2000|    0.0|
|Afghanistan|2001|    0.0|
|Afghanistan|2002|233.433|
|Afghanistan|2003|233.755|
|Afghanistan|2004|254.259|
|Afghanistan|2005|294.396|
|Afghanistan|2006|320.674|
|Afghanistan|2007|381.502|
|Afghanistan|2008|447.746|
|Afghanistan|2009|511.374|
|Afghanistan|2010| 631.49|
|Afghanistan|2011|  714.7|
|Afghanistan|2012|784.611|
|Afghanistan|2013|754.402|
|Afghanistan|2014|746.922|
|Afghanistan|2015|705.597|
|Afghanistan|2016|617.126|
|Afghanistan|2017|635.789|
|Afghanistan|2018|582.323|
|Afghanistan|2019|586.204|
+-----------+----+-------+
only showing top 20 rows

